In [4]:
!pip install pyorthanc
!pip install openpyxl



[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 8.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd 


In [51]:
# Load biopsie ergebnisse and reconstruct the correct StudyDate

biopsy_df = pd.read_excel("Biopsiebefunde zu StudyInstandUI.xlsx")
dicom_meta_data = pd.read_csv("meta_pacs_inst.csv", sep=";")
merged = pd.merge(biopsy_df, dicom_meta_data, on="StudyInstanceUID", how='left').drop(columns=['PatientName', 'StudyDate_x'])
merged = merged[~merged["StudyDate_y"].isna()]

In [72]:
# Load existing/segmented lesion cases

dicom_meta_data = dicom_meta_data[~dicom_meta_data["StudyDate"].isna()]
dicom_meta_data["StudyDate"] = dicom_meta_data["StudyDate"].astype(int).astype(str)
upladed_2023 = dicom_meta_data[dicom_meta_data["StudyDate"].str.startswith("2023")]

In [73]:
lesion_dataset = pd.read_csv("/data/oleksii/datasets/ALTA-Lesions-Dataset/segmentation/lesion_dataset.csv", sep=";")
segmented_lesions = lesion_dataset['relative_path'].unique()

In [74]:
merged["ALTA ID str"] = merged["ALTA ID"].astype(str).str.rjust(12, "0")
merged["PatientID_MRI"] = merged["PatientID_MRI"].astype(str).str.rjust(12, "0")
merged["StudyDate_y"] = merged["StudyDate_y"].astype(int).astype(str)

In [75]:
merged['relative_path'] = merged.apply(lambda row: '/'.join([row["ALTA ID str"], row["StudyDate_y"]]), axis=1)
merged['relative_path2'] = merged.apply(lambda row: '/'.join([row["PatientID_MRI"], row["StudyDate_y"]]), axis=1)


In [91]:
# make sure for account for mri id just in case
merged['segmented'] = merged['relative_path'].isin(segmented_lesions)
merged['segmented2'] = merged['relative_path2'].isin(segmented_lesions)
merged['to_exclude'] = merged['segmented'] | merged['segmented2']

In [100]:
# Remove the cases
merged = merged[~merged['StudyInstanceUID'].isin(upladed_2023['StudyInstanceUID'])]
merged = merged[~merged['to_exclude']]

In [102]:
merged = merged.drop(columns=['segmented', 'segmented2', 'seg', 'to_exclude'])

In [106]:
merged.reset_index()
merged.to_csv('biopsy_findings_to_segment_20230505.csv', index=False, sep=";")


In [107]:
len(merged)

3549